In [31]:
import dynet as dy
import numpy as np
from collections import defaultdict
from itertools import count
import sys
import random

LAYERS = 1
INPUT_DIM = 30
HIDDEN_DIM = 50

builder_type = dy.SimpleRNNBuilder
#builder = dy.LSTMBuilder
trainer_type = dy.SimpleSGDTrainer

characters = list("abcdefghijklmnopqrstuvwxyz,. ")
characters.append("<EOS>")

int2char = list(characters)
char2int = {c:i for i,c in enumerate(characters)}

VOCAB_SIZE = len(characters)

In [32]:
# return compute loss of RNN for one sentence
def do_one_sentence(rnn, params, sentence):
    # setup the sentence
    dy.renew_cg()
    s0 = rnn.initial_state()
    R = dy.parameter(params["R"])
    bias = dy.parameter(params["bias"])
    lookup = params["lookup"]
    sentence = ["<EOS>"] + list(sentence) + ["<EOS>"]
    sentence = [char2int[c] for c in sentence]
    s = s0
    loss = []
    for char,next_char in zip(sentence,sentence[1:]):
        s = s.add_input(lookup[char])
        probs = R*s.output() + bias
        loss.append(dy.pickneglogsoftmax(probs,next_char))
    loss = dy.esum(loss)
    return loss

# generate from model:
def generate(rnn, params):
    def sample(probs):
        rnd = random.random()
        for i,p in enumerate(probs):
            rnd -= p
            if rnd <= 0: break
        return i
    # setup the sentence
    dy.renew_cg()
    s0 = rnn.initial_state()
    
    R = dy.parameter(params["R"])
    bias = dy.parameter(params["bias"])
    lookup = params["lookup"]
    
    s = s0.add_input(lookup[char2int["<EOS>"]])
    out=[]
    while True:
        probs = dy.softmax(R*s.output() + bias)
        probs = probs.vec_value()
        next_char = sample(probs)
        out.append(int2char[next_char])
        if out[-1] == "<EOS>": break
        s = s.add_input(lookup[next_char])
    return "".join(out[:-1]) # strip the <EOS>

# train, and generate every 5 samples
def train(rnn, params, sentence):
    trainer = trainer_type(pc)
    for i in range(200):
        loss = do_one_sentence(rnn, params, sentence)
        loss_value = loss.value()
        loss.backward()
        trainer.update()
        if i % 5 == 0: 
            print("%.10f" % loss_value, end="\t")
            print(generate(rnn, params))

In [33]:
pc = dy.ParameterCollection()
rnn = builder_type(LAYERS, INPUT_DIM, HIDDEN_DIM, pc)
# add parameters for the hidden->output part for both lstm and srnn
params["lookup"] = pc.add_lookup_parameters((VOCAB_SIZE, INPUT_DIM))
params["R"] = pc.add_parameters((VOCAB_SIZE, HIDDEN_DIM))
params["bias"] = pc.add_parameters((VOCAB_SIZE))


In [34]:
sentence = "blaukraut bleibt blaukraut und brautkleid bleibt brautkleid"
train(rnn, params, sentence)

201.6772308350	haqbapuzwxrndh.doq,rxvtrszuizpikn,p.wesbbzdz blnonndkknultn,rfras
130.1022491455	arlci,zzfmrbkhohjwc.craorlwzbeeekbkdghfznalubbg.uwz cmtxilqa.tronvxbwsdjvb
79.8216171265	da sldidl.ykxalml pd,musbh ,btkeiazrhl
47.5023460388	iobleibt tfmlqjbblait blautkraed brwu.k rldtb leid bles  bbaahk,uw db bfaiptausqrudt zleib furautkexi  breitz brautk
27.3970603943	blaukrluwau klmukmnukeunx brautdaeidubkautrleidb brautt braudklujd bldubraut btaubrautklisd bld
17.3282775879	rlaukraet und brautkleidu bleibtjbleibtfblaukrlutfubr .skifk 
12.9416952133	bdaukraut bneibt bleibt bleibt bleibt blaukraut bleibt blaukraut bleibt blaukraut bleibt brautkleid
9.9562778473	blaukraut ulb buact bleibt brautkleid
7.5746626854	blaukraut bleibt brautkleid
7.1342582703	blaukraut und brautkleidjbleiau blaukraut und brautkleid
4.5268497467	
4.1543378830	blaucraut bleibt blaukraut und brautkyeid
2.8694839478	blaukraut bleibt blaukraut und brautkleid bleibt brauttleid bleibt brautkleid bleibt brautkleid bleib